In [3]:
#train test split as 75 25%
import os
import shutil
from sklearn.model_selection import train_test_split

def split_data(source_directory, destination_directory, train_size=0.75):
    classes = os.listdir(source_directory)
    train_directory = os.path.join(destination_directory, 'train')
    test_directory = os.path.join(destination_directory, 'test')
    
    os.makedirs(train_directory, exist_ok=True)
    os.makedirs(test_directory, exist_ok=True)
    
    for class_name in classes:
        class_dir = os.path.join(source_directory, class_name)
        train_class_dir = os.path.join(train_directory, class_name)
        test_class_dir = os.path.join(test_directory, class_name)
        
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
        
        all_files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        train_files, test_files = train_test_split(all_files, train_size=train_size, random_state=42)
        
        for file in train_files:
            shutil.move(os.path.join(class_dir, file), os.path.join(train_class_dir, file))
        for file in test_files:
            shutil.move(os.path.join(class_dir, file), os.path.join(test_class_dir, file))

source_dir = 'D:\Datasets\\InsectClassification\\TrainSet\\augmented_farm_insects\\'
dest_dir = 'D:\\Datasets\\split_farm_insects\\'

split_data(source_dir, dest_dir)


In [10]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing.image import save_img
import random

source_dir = r"D:\Datasets\Newhammad\ResizedTrainingBgRemoved"
dest_dir = r"D:\Datasets\Newhammad\Augmented"

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

def augment_images(class_name, image_list):
    """Augment images in the specified class folder and save them."""
    dest_class_dir = os.path.join(dest_dir, class_name)
    os.makedirs(dest_class_dir, exist_ok=True)

    for img_name in image_list:
        img_path = os.path.join(source_dir, class_name, img_name)
        img = load_img(img_path)
        img_array = img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)
        
        i = 0
        for _ in range(4): 
            aug_iter = datagen.flow(img_array, batch_size=1)
            aug_img = next(aug_iter)[0].astype('uint8')
            
            new_img_path = os.path.join(dest_class_dir, f"aug_{i}_{img_name}")
            save_img(new_img_path, aug_img)
            i += 1
        
        original_img_dest_path = os.path.join(dest_class_dir, img_name)
        img.save(original_img_dest_path)

for class_name in os.listdir(source_dir):
    class_dir = os.path.join(source_dir, class_name)
    image_list = os.listdir(class_dir)
    
    if os.path.isdir(class_dir):
        augment_images(class_name, image_list)

print("Augmentation completed.")


Augmentation completed.


In [11]:
#augmentation of images

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='reflect'
)

def augment_images(input_directory, output_directory, target_size=(256, 256), batch_size=32, target_count=400):
    for class_dir in os.listdir(input_directory):
        input_class_path = os.path.join(input_directory, class_dir)
        output_class_path = os.path.join(output_directory, class_dir)
        os.makedirs(output_class_path, exist_ok=True)
        
        total_images_needed = target_count
        num_original_images = len([name for name in os.listdir(input_class_path) if os.path.isfile(os.path.join(input_class_path, name))])
        num_images_to_generate = total_images_needed - num_original_images

        if num_images_to_generate > 0:  # Only augment if there are less than 400 images already
            generator = datagen.flow_from_directory(
                directory=input_directory,
                classes=[class_dir],
                target_size=target_size,
                batch_size=batch_size,
                save_to_dir=output_class_path,
                save_format='jpeg',
                save_prefix='aug_'
            )

            num_batches_needed = (num_images_to_generate + batch_size - 1) // batch_size

            for _ in range(21):
                next(generator)

input_dir = 'D:\\Datasets\\split_farm_insects\\train\\'
output_dir = 'D:\\Datasets\\augmented_farm_insects\\'
augment_images(input_dir, output_dir)



Found 71 images belonging to 1 classes.
Found 66 images belonging to 1 classes.
Found 71 images belonging to 1 classes.
Found 85 images belonging to 1 classes.
Found 78 images belonging to 1 classes.
Found 78 images belonging to 1 classes.
Found 84 images belonging to 1 classes.
Found 85 images belonging to 1 classes.
Found 82 images belonging to 1 classes.
Found 82 images belonging to 1 classes.
Found 74 images belonging to 1 classes.
Found 88 images belonging to 1 classes.
Found 81 images belonging to 1 classes.
Found 81 images belonging to 1 classes.
Found 72 images belonging to 1 classes.


In [11]:
#training data csv
import os
import csv


base_path = "D:\\Datasets\\Newhammad\\Augmented"

output_csv_file = "D:\\Datasets\\Newhammad\\Augmented\\image_labels.csv"

with open(output_csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "Label"])  

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    writer.writerow([filename, folder])
                else:
                    print(filename,folder,"nignog")

print("CSV file has been created successfully.")


CSV file has been created successfully.


In [5]:
#testing data csv
import os
import csv

base_path = "D:\\Datasets\\Newhammad\\AugmentedTest"

output_csv_file = "D:\\Datasets\\Newhammad\\AugmentedTest\\test_labels.csv"

with open(output_csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "Label"])  

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    writer.writerow([filename, folder])
                else:
                    print(filename,folder,"nignog")

print("CSV file has been created successfully.")


aug_0_Image_115.gif Aphids nignog
Image_115.gif Aphids nignog
aug_0_Image_40.gif Fall Armyworms nignog
Image_40.gif Fall Armyworms nignog
CSV file has been created successfully.


In [12]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set")
    except RuntimeError as e:
        print(e)




Num GPUs Available:  1
Memory growth set


In [1]:
import os
from PIL import Image
from rembg import remove

def remove_background_from_folder(input_folder, output_folder):
    for class_name in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_name)
        
        if os.path.isdir(class_path):
            output_class_path = os.path.join(output_folder, class_name)
            if not os.path.exists(output_class_path):
                os.makedirs(output_class_path)

            for image_name in os.listdir(class_path):
                input_image_path = os.path.join(class_path, image_name)
                output_image_path = os.path.join(output_class_path, image_name)

                try:
                    input_image = Image.open(input_image_path)
                    output_image = remove(input_image)

                    if output_image.mode == 'RGBA':
                        output_image = output_image.convert('RGB')
                    
                    output_image.save(output_image_path)
                    print(f"Processed {image_name} successfully.")
                except Exception as e:
                    print(f"Failed to process {image_name}: {str(e)}")


input_folder_train="D:\\Datasets\\InsectClassification\\split_farm_insects\\train"
output_folder_test="D:\\RemovedDataset\\Train"
input_folder_test = "D:\\Datasets\\InsectClassification\\split_farm_insects\\test"
output_folder_test = "D:\\RemovedDataset\\Test"


remove_background_from_folder(input_folder_train, output_folder_test)
remove_background_from_folder(input_folder_test, output_folder_test)


Processed Image_1.jpg successfully.
Processed Image_104.jpg successfully.
Processed Image_110.jpg successfully.
Processed Image_117.jpg successfully.
Processed Image_13.jpg successfully.
Processed Image_16.jpg successfully.
Processed Image_20.jpg successfully.
Processed Image_24.jpg successfully.
Processed Image_29.jpg successfully.
Processed Image_36.jpg successfully.
Processed Image_37.png successfully.
Processed Image_39.jpg successfully.
Processed Image_40.jpg successfully.
Processed Image_43.jpg successfully.
Processed Image_45.jpg successfully.
Processed Image_59.jpg successfully.
Processed Image_60.jpg successfully.
Processed Image_64.jpg successfully.
Processed Image_67.jpg successfully.
Processed Image_7.jpg successfully.
Processed Image_71.jpg successfully.
Processed Image_8.jpg successfully.
Processed Image_81.jpg successfully.
Processed Image_87.jpg successfully.
Processed Image_92.jpg successfully.
Processed Image_1.jpeg successfully.
Processed Image_102.jpg successfully.


C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Processed Image_14.png successfully.
Processed Image_19.png successfully.
Processed Image_23.jpg successfully.
Processed Image_27.jpg successfully.
Processed Image_30.jpg successfully.
Processed Image_31.jpg successfully.
Processed Image_33.jpg successfully.
Processed Image_39.jpg successfully.
Processed Image_50.jpg successfully.
Processed Image_51.jpg successfully.
Processed Image_53.jpg successfully.
Processed Image_54.jpg successfully.
Processed Image_56.jpg successfully.
Processed Image_66.jpg successfully.
Processed Image_67.jpg successfully.
Processed Image_69.jpg successfully.
Processed Image_77.jpg successfully.
Processed Image_84.jpg successfully.
Processed Image_9.jpg successfully.
Processed Image_94.jpg successfully.
Processed Image_97.jpg successfully.
Processed Image_98.jpg successfully.
Processed Image_1.jpg successfully.
Processed Image_102.jpg successfully.
Processed Image_108.jpg successfully.
Processed Image_11.jpg successfully.
Processed Image_115.jpg successfully.


In [24]:
from skimage.io import imread
from skimage.transform import resize
from skimage.color import gray2rgb
import numpy as np
import pandas as pd
import os

def load_images(dataframe, base_path):
    images = []
    labels = []
    for idx, row in dataframe.iterrows():
        img_path = os.path.join(base_path, row['Label'], row['Image Name'])
        img = imread(img_path)
        if img.ndim == 2: 
            img = gray2rgb(img)
        elif img.shape[2] == 4: 
            img = img[:, :, :3]
        img = resize(img, (224, 224), anti_aliasing=True)
        images.append(img.astype(np.float32)) 
        labels.append(row['Label'])
    return np.array(images), labels  


train_df = pd.read_csv('D:\\Datasets\\Newhammad\\Augmented\\image_labels.csv')
test_df = pd.read_csv('D:\\Datasets\\Newhammad\\AugmentedTest\\test_labels.csv')

train_images, train_labels = load_images(train_df, 'D:\\Datasets\\Newhammad\\Augmented')
test_images, test_labels = load_images(test_df, 'D:\\Datasets\\Newhammad\\AugmentedTest')

print("Train images shape:", train_images.shape)
print("Test images shape:", test_images.shape)



Train images shape: (5250, 224, 224, 3)
Test images shape: (782, 224, 224, 3)


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(set(train_labels)), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)  
x = Activation('relu')(x) 
x = Dropout(0.5)(x)  
predictions = Dense(len(set(train_labels)), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np

le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
train_labels_enc = to_categorical(train_labels_enc)

model.fit(np.array(train_images), np.array(train_labels_enc), epochs=15, batch_size=16)

Epoch 1/15
329/329 [==============================] - 49s 121ms/step - loss: 1.6533 - accuracy: 0.4735
Epoch 2/15
329/329 [==============================] - 42s 129ms/step - loss: 0.8777 - accuracy: 0.7084
Epoch 3/15
329/329 [==============================] - 39s 120ms/step - loss: 0.4980 - accuracy: 0.8370
Epoch 4/15
329/329 [==============================] - 38s 116ms/step - loss: 0.3165 - accuracy: 0.8983
Epoch 5/15
329/329 [==============================] - 38s 117ms/step - loss: 0.2068 - accuracy: 0.9301
Epoch 6/15
329/329 [==============================] - 39s 119ms/step - loss: 0.1501 - accuracy: 0.9541
Epoch 7/15
329/329 [==============================] - 39s 120ms/step - loss: 0.1269 - accuracy: 0.9608
Epoch 8/15
329/329 [==============================] - 40s 123ms/step - loss: 0.0932 - accuracy: 0.9695
Epoch 9/15
329/329 [==============================] - 39s 118ms/step - loss: 0.0684 - accuracy: 0.9789
Epoch 10/15
329/329 [==============================] - 41s 124ms/step - l

In [25]:
from sklearn.metrics import classification_report

le.fit(train_labels)

test_labels_enc = le.transform(test_labels)

filtered_test_images = test_images[test_labels_enc != -1]
filtered_test_labels_enc = test_labels_enc[test_labels_enc != -1]

test_predictions = model.predict(filtered_test_images)
predicted_labels = np.argmax(test_predictions, axis=1)

class_report = classification_report(filtered_test_labels_enc, predicted_labels, target_names=le.classes_)
print("Classification Report:")
print(class_report)

25/25 [==============================] - 1s 56ms/step
Classification Report:
                                      precision    recall  f1-score   support

Africanized Honey Bees (Killer Bees)       1.00      0.66      0.80        50
                              Aphids       0.94      0.74      0.83        42
                           Armyworms       0.57      0.67      0.62        48
         Brown Marmorated Stink Bugs       0.94      0.81      0.87        58
                     Cabbage Loopers       1.00      0.72      0.84        46
                       Citrus Canker       0.77      0.85      0.81        52
             Colorado Potato Beetles       0.96      0.89      0.93        56
                         Corn Borers       0.73      0.70      0.72        54
                       Corn Earworms       0.97      0.70      0.81        50
                      Fall Armyworms       0.63      0.54      0.58        54
                         Fruit Flies       0.89      0.82      0

In [26]:
model.save('ClassifierInsects.h5')